In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/visML_project/Data/sensor_1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# generate frames from left.mp4
#!ffmpeg -i left.mp4  img1/%06d.jpg

In [22]:
ls

array.json  left_half.mp4  left.mp4          left_quarter.mp4  right.json  right_people.json
img1/       left.json      left_people.json  right_half.mp4    right.mp4   right_quarter.mp4


In [21]:
#%ls img1/*.jpg | wc -l
# 40395

40395


In [ ]:
import json
with open('left_people.json', 'r') as file:
    left_people = json.load(file)

In [40]:
# remove the empty elements in left_people.json and the corresponding frame in img1
# rename frame names in img1 to maintain continuous numbering
indices_to_remove = [i for i, person in enumerate(left_people) if person is None]
left_people_cleaned = [person for i, person in enumerate(left_people) if i not in indices_to_remove]
with open('left_people_cleaned.json', 'w') as file:
    json.dump(left_people_cleaned, file)

In [41]:
with open('left_people_cleaned.json', 'r') as file:
    left_people_cleaned = json.load(file)

In [43]:
len(left_people_cleaned)

39617

In [ ]:
img_dir = 'img1/'
# Delete corresponding frames
for index in indices_to_remove:
    img_filename = f"{index + 1:06d}.jpg"
    img_path = os.path.join(img_dir, img_filename)
    if os.path.exists(img_path):
        os.remove(img_path)
        print(f"Deleted {img_path}")


In [48]:
%ls img1/*.jpg | wc -l

39617


In [46]:
remaining_images = sorted(os.listdir(img_dir))
new_index = 1
# Rename the remaining images
for old_name in remaining_images:
    new_name = f"{new_index:06d}.jpg"
    old_path = os.path.join(img_dir, old_name)
    new_path = os.path.join(img_dir, new_name)
    os.rename(old_path, new_path)
    print(f"Renamed {old_path} to {new_path}")
    new_index += 1

Streaming output truncated to the last 5000 lines.
Renamed img1/034973.jpg to img1/034618.jpg
Renamed img1/034974.jpg to img1/034619.jpg
Renamed img1/034975.jpg to img1/034620.jpg
Renamed img1/034976.jpg to img1/034621.jpg
Renamed img1/034977.jpg to img1/034622.jpg
Renamed img1/034978.jpg to img1/034623.jpg
Renamed img1/034979.jpg to img1/034624.jpg
Renamed img1/034980.jpg to img1/034625.jpg
Renamed img1/034981.jpg to img1/034626.jpg
Renamed img1/034982.jpg to img1/034627.jpg
Renamed img1/034983.jpg to img1/034628.jpg
Renamed img1/034984.jpg to img1/034629.jpg
Renamed img1/034985.jpg to img1/034630.jpg
Renamed img1/034986.jpg to img1/034631.jpg
Renamed img1/034987.jpg to img1/034632.jpg
Renamed img1/034988.jpg to img1/034633.jpg
Renamed img1/034989.jpg to img1/034634.jpg
Renamed img1/034990.jpg to img1/034635.jpg
Renamed img1/034991.jpg to img1/034636.jpg
Renamed img1/034992.jpg to img1/034637.jpg
Renamed img1/034993.jpg to img1/034638.jpg
Renamed img1/034994.jpg to img1/034639.jpg
Ren

In [49]:
# convert the left_people_cleaned to the mot format
import csv
with open('left_people_cleaned_mot.txt', 'w', newline='') as txtfile:
    writer = csv.writer(txtfile, delimiter=',')

    for frame_index, frame_data in enumerate(left_people_cleaned):
        if frame_data is not None:
            for detection_index, box in enumerate(frame_data.get('boxes', [])):
                try:
                    bb_top_left = box[0]
                    bb_bottom_right = box[1]
                    bb_left = bb_top_left[0]
                    bb_top = bb_top_left[1]
                    bb_width = bb_bottom_right[0] - bb_left
                    bb_height = bb_bottom_right[1] - bb_top
                    conf = frame_data['scores'][detection_index] if 'scores' in frame_data and len(frame_data['scores']) > detection_index else -1

                    writer.writerow([
                        frame_index + 1, # <frame>
                        -1,  # <id>
                        bb_left, bb_top, bb_width, bb_height, #<bb_left>,<bb_top>,<bb_width>,<bb_height>
                        conf,  # <conf>
                        -1, -1, -1 # <x>, <y>,<z>
                    ])
                except IndexError:
                    print(f"Error processing data for frame {frame_index+1}, detection {detection_index+1}")
        else:
            writer.writerow([frame_index + 1, -1, -1, -1, -1, -1, -1, -1, -1,-1])


In [51]:
import csv
with open('left_people_cleaned_mot.txt', 'r') as file:
    reader = csv.reader(file)
    left_people_cleaned_mot = list(reader)

In [56]:
print(len(left_people_cleaned_mot))
print(left_people_cleaned_mot[0:5])
print(left_people_cleaned_mot[-4:])

213665
[['1', '-1', '1410', '376', '78', '233', '0.3655892610549927', '-1', '-1', '-1'], ['1', '-1', '1578', '359', '97', '262', '0.3712770938873291', '-1', '-1', '-1'], ['1', '-1', '2159', '299', '49', '179', '0.3223310708999634', '-1', '-1', '-1'], ['1', '-1', '724', '445', '94', '192', '0.6393248438835144', '-1', '-1', '-1'], ['1', '-1', '2051', '299', '68', '200', '0.3606260418891907', '-1', '-1', '-1']]
[['39617', '-1', '1512', '392', '144', '302', '0.46360063552856445', '-1', '-1', '-1'], ['39617', '-1', '1732', '332', '53', '156', '0.37357020378112793', '-1', '-1', '-1'], ['39617', '-1', '2096', '300', '83', '197', '0.6163380742073059', '-1', '-1', '-1'], ['39617', '-1', '2162', '291', '81', '200', '-1', '-1', '-1', '-1']]


In [59]:
left_people_cleaned[-1]['boxes']

[[[1512, 392], [1656, 694]],
 [[1732, 332], [1785, 488]],
 [[2096, 300], [2179, 497]],
 [[2162, 291], [2243, 491]]]

In [62]:
!ffmpeg -i left.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [66]:
import configparser

# Create a configparser object
config = configparser.ConfigParser()

# Add the sequence section
config['Sequence'] = {
    'name': 'sensor_1_left',
    'imDir': 'img1',
    'frameRate': 14.40,
    'seqLength': 39617,
    'imWidth': 2592,
    'imHeight': 1944,
    'imExt': '.jpg'
}

with open('seqinfo.ini', 'w') as configfile:
    config.write(configfile)


In [67]:
ls

array.json     left.mp4                     left_quarter.mp4  right_people.json
img1/          left_people_cleaned.json     right_half.mp4    right_quarter.mp4
left_half.mp4  left_people_cleaned_mot.txt  right.json        seqinfo.ini
left.json      left_people.json             right.mp4
